In [2]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm

link='C:/Users/NavneethBhat/Desktop/Code/Implementations/data/train.csv/train.csv'

class DigitDataset(Dataset):
    def __init__(self,link):
        dataset=pd.read_csv(link,dtype=np.float32)
        y=dataset['label'].values
        x=dataset.drop('label',axis=1).values
        self.x=torch.from_numpy(x)
        self.y=torch.from_numpy(y)
        
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
    def __len__(self):
        return self.x.shape[0]
    
data=DigitDataset(link)


device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes=10
num_epochs=2
batch_size=1
learning_rate=2e-5

train_loader=DataLoader(dataset=data,batch_size=batch_size,shuffle=True)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=nn.Conv2d(1,6,3)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(6*13*13,10)
    
    def forward(self,x):
        x=self.pool(self.c1(x))
        x=torch.flatten(x,1)
       # print(x.shape)
        x=self.fc1(x)
       # print(x.shape)
        return x

In [4]:
model=Net()
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

#Sets the learning rate of each parameter group to the initial lr times a given function.
l1=lambda epoch: epoch/10
scheduler=torch.optim.lr_scheduler.LambdaLR(optimizer,l1)

#Multiply the learning rate of each parameter group by the factor given in the specified function.
l2=lambda epoch:0.95
#scheduler=torch.optim.lr_scheduler.MultiplicativeLR(optimizer,l2)

#Decays the learning rate of each parameter group by gamma every step_size epochs. 
#torch.optim.lr_scheduler.StepLR(optimizer, 2, gamma=0.1)

In [5]:
for epoch in range(num_epochs):
    #create progress bar with tqdm
    #enumerate is basically a counter which allows you to access the batch 
    for i, (images,labels) in tqdm(enumerate(train_loader),total=len(train_loader),leave=False):
        labels=labels.type(torch.LongTensor)
        images=images.view(1,1,28,28)
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        preds=model(images)
        #print(type(preds))
        #print(labels.shape)
        loss=criterion(preds,labels)
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(optimizer.state_dict()['param_groups'][0]['lr'])    

    print(loss)

2.0000000000000003e-06
tensor(61.1043, device='cuda:0', grad_fn=<NllLossBackward0>)


4.000000000000001e-06
tensor(0.8663, device='cuda:0', grad_fn=<NllLossBackward0>)


In [20]:
import matplotlib.pyplot as plt
image,label=next(iter(train_loader))
with torch.no_grad():
    image=image.view(1,1,28,28)
    image=image.to(device)
    pred=model(image)
    print(torch.argmax(pred))
    print(label)

tensor(3, device='cuda:0')
tensor([3.])
